In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [1]:
%%capture
!conda install -c conda-forge openmm mdtraj nglview

In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout, argv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md

In [6]:
pdb = PDBFile("1yrf_a.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model="tip3p", padding=1 * nanometer)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
positions = simulation.context.getState(getPositions=True).getPositions()
with open("init.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [7]:
simulation.reporters = []
simulation.reporters.append(DCDReporter("traj.dcd", 1000))
simulation.reporters.append(
    StateDataReporter(stdout, 100, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars.csv",
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
    )
)
simulation.step(1e6)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

Streaming output truncated to the last 5000 lines.
500100,311.34776679894895,187.98484110832214
500200,307.63182893406076,188.01498913764954
500300,301.60698169309524,188.0457046031952
500400,305.8831763880039,188.07657122612
500500,301.1188624708563,188.10824537277222
500600,302.1735632313885,188.13797807693481
500700,301.8950445163516,188.16937971115112
500800,297.8505655564563,188.20026445388794
500900,305.08661774256734,188.23149394989014
501000,306.4490933225715,188.29947805404663
501100,304.2794912552037,188.32818388938904
501200,301.22581247107684,188.35731053352356
501300,299.6411721591083,188.38767743110657
501400,301.56872172850785,188.41852140426636
501500,306.8008433445031,188.45241165161133
501600,297.7142493085714,188.4825143814087
501700,296.48918093270794,188.51393222808838
501800,299.66624497141754,188.54393815994263
501900,301.94550194407253,188.57505583763123
502000,295.36967845585013,188.64155220985413
502100,294.8821017664991,188.6738932132721
502200,295.7001449346